# 03. 条件边和决策逻辑

## 课程目标
- 掌握条件边（Conditional Edges）的使用
- 学习编写决策函数
- 实现多分支路由逻辑
- 构建循环和递归结构
- 理解动态路径选择机制

## 核心概念

条件边是 LangGraph 中的重要特性，它允许根据状态内容动态决定执行路径：
1. **决策函数**：根据状态返回路径标识
2. **路径映射**：将决策结果映射到具体节点
3. **多路分支**：支持复杂的分支逻辑
4. **循环控制**：实现条件循环和递归

## 1. 环境准备

In [ ]:
from typing import TypedDict, Literal, Optional, Dict, List
from langgraph.graph import StateGraph, END
import random
import json
from IPython.display import Image, display

## 2. 基础条件边示例

让我们从一个简单的条件边示例开始：

In [ ]:
# 定义状态
class SimpleState(TypedDict):
    number: int
    result: str
    path: str

# 创建图
simple_graph = StateGraph(SimpleState)

def generate_number(state: SimpleState) -> SimpleState:
    """生成随机数"""
    num = random.randint(1, 100)
    print(f"生成数字: {num}")
    return {
        "number": num,
        "result": f"生成了数字 {num}",
        "path": "generated"
    }

def process_even(state: SimpleState) -> SimpleState:
    """处理偶数"""
    num = state["number"]
    result = num // 2
    print(f"处理偶数 {num}: {num} ÷ 2 = {result}")
    return {
        "result": f"偶数 {num} 处理结果: {result}",
        "path": "even_processed"
    }

def process_odd(state: SimpleState) -> SimpleState:
    """处理奇数"""
    num = state["number"]
    result = num * 3 + 1
    print(f"处理奇数 {num}: {num} × 3 + 1 = {result}")
    return {
        "result": f"奇数 {num} 处理结果: {result}",
        "path": "odd_processed"
    }

def decide_path(state: SimpleState) -> Literal["even", "odd"]:
    """决策函数：根据数字奇偶性决定路径"""
    num = state["number"]
    if num % 2 == 0:
        print(f"数字 {num} 是偶数，选择偶数处理路径")
        return "even"
    else:
        print(f"数字 {num} 是奇数，选择奇数处理路径")
        return "odd"

In [ ]:
# 构建简单条件图
simple_graph.add_node("generate", generate_number)
simple_graph.add_node("process_even", process_even)
simple_graph.add_node("process_odd", process_odd)

# 设置入口点
simple_graph.set_entry_point("generate")

# 添加条件边
simple_graph.add_conditional_edges(
    "generate",  # 从哪个节点出发
    decide_path,  # 决策函数
    {
        "even": "process_even",  # 路径映射
        "odd": "process_odd"
    }
)

# 两个处理节点都连接到END
simple_graph.add_edge("process_even", END)
simple_graph.add_edge("process_odd", END)

# 编译
simple_app = simple_graph.compile()

# 测试多次执行
for i in range(3):
    print(f"\n=== 第 {i+1} 次执行 ===")
    result = simple_app.invoke({})
    print(f"最终结果: {result['result']}")

## 3. 多路分支决策

条件边支持更复杂的多路分支决策：

In [ ]:
# 多路分支状态
class MultiPathState(TypedDict):
    score: int
    grade: str
    message: str
    actions: List[str]

# 创建多路分支图
multi_graph = StateGraph(MultiPathState)

def calculate_score(state: MultiPathState) -> MultiPathState:
    """计算分数"""
    score = random.randint(0, 100)
    print(f"🎯 计算得分: {score}")
    return {
        "score": score,
        "message": f"获得分数: {score}",
        "actions": ["计算分数"]
    }

def excellent_path(state: MultiPathState) -> MultiPathState:
    """优秀路径"""
    print("🌟 进入优秀路径")
    return {
        "grade": "A",
        "message": "恭喜！您的表现非常优秀！",
        "actions": state["actions"] + ["优秀认定", "发放奖励"]
    }

def good_path(state: MultiPathState) -> MultiPathState:
    """良好路径"""
    print("👍 进入良好路径")
    return {
        "grade": "B",
        "message": "表现良好，继续保持！",
        "actions": state["actions"] + ["良好认定", "鼓励继续"]
    }

def average_path(state: MultiPathState) -> MultiPathState:
    """一般路径"""
    print("📈 进入一般路径")
    return {
        "grade": "C",
        "message": "还有提升空间，需要努力！",
        "actions": state["actions"] + ["一般认定", "建议改进"]
    }

def poor_path(state: MultiPathState) -> MultiPathState:
    """较差路径"""
    print("🔄 进入较差路径")
    return {
        "grade": "D",
        "message": "需要重新学习和改进。",
        "actions": state["actions"] + ["较差认定", "安排补课"]
    }

def determine_grade(state: MultiPathState) -> Literal["excellent", "good", "average", "poor"]:
    """根据分数确定等级"""
    score = state["score"]
    
    if score >= 90:
        print(f"分数 {score} >= 90，等级：优秀")
        return "excellent"
    elif score >= 80:
        print(f"分数 {score} >= 80，等级：良好")
        return "good"
    elif score >= 60:
        print(f"分数 {score} >= 60，等级：一般")
        return "average"
    else:
        print(f"分数 {score} < 60，等级：较差")
        return "poor"

In [ ]:
# 构建多路分支图
multi_graph.add_node("calculate", calculate_score)
multi_graph.add_node("excellent", excellent_path)
multi_graph.add_node("good", good_path)
multi_graph.add_node("average", average_path)
multi_graph.add_node("poor", poor_path)

multi_graph.set_entry_point("calculate")

# 添加多路条件边
multi_graph.add_conditional_edges(
    "calculate",
    determine_grade,
    {
        "excellent": "excellent",
        "good": "good",
        "average": "average",
        "poor": "poor"
    }
)

# 所有路径都连接到END
multi_graph.add_edge("excellent", END)
multi_graph.add_edge("good", END)
multi_graph.add_edge("average", END)
multi_graph.add_edge("poor", END)

# 编译和测试
multi_app = multi_graph.compile()

# 测试多次执行
for i in range(5):
    print(f"\n{'='*30} 测试 {i+1} {'='*30}")
    result = multi_app.invoke({})
    print(f"等级: {result['grade']} | {result['message']}")
    print(f"执行的操作: {' -> '.join(result['actions'])}")

## 4. 循环和递归结构

条件边可以创建循环结构，实现重复执行直到满足某个条件：

In [ ]:
# 循环状态定义
class LoopState(TypedDict):
    current_number: int
    target: int
    attempts: int
    max_attempts: int
    history: List[int]
    status: str

# 创建循环图
loop_graph = StateGraph(LoopState)

def initialize(state: LoopState) -> LoopState:
    """初始化"""
    target = random.randint(50, 100)
    print(f"🎯 初始化：目标数字是 {target}")
    return {
        "current_number": 1,
        "target": target,
        "attempts": 0,
        "max_attempts": 10,
        "history": [1],
        "status": "initialized"
    }

def generate_guess(state: LoopState) -> LoopState:
    """生成猜测"""
    current = state["current_number"]
    attempts = state["attempts"] + 1
    
    # 简单的猜测策略：随机增加一个数
    increment = random.randint(1, 15)
    new_number = current + increment
    
    history = state["history"] + [new_number]
    
    print(f"🔢 第 {attempts} 次尝试: {current} + {increment} = {new_number}")
    
    return {
        "current_number": new_number,
        "attempts": attempts,
        "history": history,
        "status": "guessing"
    }

def check_result(state: LoopState) -> LoopState:
    """检查结果"""
    current = state["current_number"]
    target = state["target"]
    attempts = state["attempts"]
    
    if current >= target:
        print(f"🎉 成功！在 {attempts} 次尝试后达到目标 {target}（当前: {current}）")
        status = "success"
    elif attempts >= state["max_attempts"]:
        print(f"❌ 失败！已达到最大尝试次数 {state['max_attempts']}")
        status = "failed"
    else:
        print(f"📊 当前: {current}, 目标: {target}, 尝试: {attempts}")
        status = "continue"
    
    return {
        "status": status
    }

def decide_continue(state: LoopState) -> Literal["continue", "end"]:
    """决定是否继续循环"""
    status = state["status"]
    
    if status == "continue":
        print("➡️ 继续尝试")
        return "continue"
    else:
        print("🛑 结束循环")
        return "end"

def finalize(state: LoopState) -> LoopState:
    """结束处理"""
    status = state["status"]
    attempts = state["attempts"]
    history = state["history"]
    
    if status == "success":
        message = f"任务完成！用了 {attempts} 次尝试"
    else:
        message = f"任务失败！尝试了 {attempts} 次"
    
    print(f"📋 {message}")
    print(f"历史记录: {' -> '.join(map(str, history))}")
    
    return {
        "status": "finalized"
    }

In [ ]:
# 构建循环图
loop_graph.add_node("initialize", initialize)
loop_graph.add_node("guess", generate_guess)
loop_graph.add_node("check", check_result)
loop_graph.add_node("finalize", finalize)

loop_graph.set_entry_point("initialize")

# 初始化后进入猜测
loop_graph.add_edge("initialize", "guess")
# 猜测后检查
loop_graph.add_edge("guess", "check")

# 关键的循环逻辑：根据检查结果决定是否继续
loop_graph.add_conditional_edges(
    "check",
    decide_continue,
    {
        "continue": "guess",  # 循环回到猜测节点
        "end": "finalize"     # 结束到最终化节点
    }
)

loop_graph.add_edge("finalize", END)

# 编译和测试
loop_app = loop_graph.compile()

print("开始循环猜数字游戏...\n")
result = loop_app.invoke({})

print(f"\n最终状态: {result['status']}")
print(f"总尝试次数: {result['attempts']}")
print(f"数字序列: {result['history']}")

## 5. 复杂决策：状态机模式

我们可以使用条件边实现复杂的状态机：

In [ ]:
from enum import Enum

# 定义状态枚举
class OrderStatus(str, Enum):
    CREATED = "created"
    VALIDATED = "validated"
    PAID = "paid"
    SHIPPED = "shipped"
    DELIVERED = "delivered"
    CANCELLED = "cancelled"
    REFUNDED = "refunded"

# 订单处理状态
class OrderState(TypedDict):
    order_id: str
    status: OrderStatus
    amount: float
    customer_id: str
    items: List[str]
    error_message: Optional[str]
    processing_log: List[str]
    retry_count: int

# 创建订单状态机
order_graph = StateGraph(OrderState)

def create_order(state: OrderState) -> OrderState:
    """创建订单"""
    order_id = f"ORD-{random.randint(1000, 9999)}"
    print(f"📝 创建订单: {order_id}")
    
    return {
        "order_id": order_id,
        "status": OrderStatus.CREATED,
        "amount": round(random.uniform(10.0, 500.0), 2),
        "customer_id": f"CUST-{random.randint(100, 999)}",
        "items": [f"商品{i}" for i in range(1, random.randint(2, 5))],
        "error_message": None,
        "processing_log": ["订单已创建"],
        "retry_count": 0
    }

def validate_order(state: OrderState) -> OrderState:
    """验证订单"""
    order_id = state["order_id"]
    
    # 模拟验证过程（90% 成功率）
    success = random.random() > 0.1
    
    if success:
        print(f"✅ 订单 {order_id} 验证通过")
        return {
            "status": OrderStatus.VALIDATED,
            "processing_log": state["processing_log"] + ["订单验证通过"]
        }
    else:
        print(f"❌ 订单 {order_id} 验证失败")
        return {
            "status": OrderStatus.CANCELLED,
            "error_message": "订单验证失败",
            "processing_log": state["processing_log"] + ["订单验证失败"]
        }

def process_payment(state: OrderState) -> OrderState:
    """处理支付"""
    order_id = state["order_id"]
    amount = state["amount"]
    
    # 模拟支付处理（85% 成功率）
    success = random.random() > 0.15
    
    if success:
        print(f"💳 订单 {order_id} 支付成功，金额: ¥{amount}")
        return {
            "status": OrderStatus.PAID,
            "processing_log": state["processing_log"] + [f"支付成功: ¥{amount}"]
        }
    else:
        print(f"💳 订单 {order_id} 支付失败")
        retry_count = state["retry_count"] + 1
        
        if retry_count >= 3:
            return {
                "status": OrderStatus.CANCELLED,
                "error_message": "支付失败，已达最大重试次数",
                "retry_count": retry_count,
                "processing_log": state["processing_log"] + [f"支付失败（第{retry_count}次）"]
            }
        else:
            return {
                "status": OrderStatus.VALIDATED,  # 回到验证状态重试
                "error_message": f"支付失败，准备重试（第{retry_count}次）",
                "retry_count": retry_count,
                "processing_log": state["processing_log"] + [f"支付失败，准备重试（第{retry_count}次）"]
            }

def ship_order(state: OrderState) -> OrderState:
    """发货"""
    order_id = state["order_id"]
    print(f"🚚 订单 {order_id} 开始发货")
    
    return {
        "status": OrderStatus.SHIPPED,
        "processing_log": state["processing_log"] + ["订单已发货"]
    }

def deliver_order(state: OrderState) -> OrderState:
    """送达"""
    order_id = state["order_id"]
    print(f"📦 订单 {order_id} 已送达")
    
    return {
        "status": OrderStatus.DELIVERED,
        "processing_log": state["processing_log"] + ["订单已送达"]
    }

def cancel_order(state: OrderState) -> OrderState:
    """取消订单"""
    order_id = state["order_id"]
    error = state.get("error_message", "未知原因")
    print(f"❌ 订单 {order_id} 已取消：{error}")
    
    return {
        "processing_log": state["processing_log"] + [f"订单已取消：{error}"]
    }

def route_order(state: OrderState) -> str:
    """订单路由决策"""
    status = state["status"]
    print(f"🔀 当前状态: {status}, 决定下一步...")
    
    # 状态转换逻辑
    transitions = {
        OrderStatus.CREATED: "validate",
        OrderStatus.VALIDATED: "pay",
        OrderStatus.PAID: "ship",
        OrderStatus.SHIPPED: "deliver",
        OrderStatus.DELIVERED: "end",
        OrderStatus.CANCELLED: "cancel"
    }
    
    return transitions.get(status, "end")

In [ ]:
# 构建订单状态机
order_graph.add_node("create", create_order)
order_graph.add_node("validate", validate_order)
order_graph.add_node("pay", process_payment)
order_graph.add_node("ship", ship_order)
order_graph.add_node("deliver", deliver_order)
order_graph.add_node("cancel", cancel_order)

order_graph.set_entry_point("create")

# 从创建开始，后续根据状态路由
order_graph.add_conditional_edges(
    "create",
    route_order,
    {
        "validate": "validate",
        "pay": "pay",
        "ship": "ship",
        "deliver": "deliver",
        "cancel": "cancel",
        "end": END
    }
)

# 每个节点执行后都重新路由
for node in ["validate", "pay", "ship", "deliver"]:
    order_graph.add_conditional_edges(
        node,
        route_order,
        {
            "validate": "validate",
            "pay": "pay",
            "ship": "ship",
            "deliver": "deliver",
            "cancel": "cancel",
            "end": END
        }
    )

order_graph.add_edge("cancel", END)

# 编译
order_app = order_graph.compile()

# 测试订单处理
for i in range(3):
    print(f"\n{'='*20} 处理订单 {i+1} {'='*20}")
    result = order_app.invoke({})
    
    print(f"\n📋 订单摘要:")
    print(f"订单ID: {result['order_id']}")
    print(f"最终状态: {result['status']}")
    print(f"金额: ¥{result['amount']}")
    print(f"重试次数: {result['retry_count']}")
    if result.get('error_message'):
        print(f"错误信息: {result['error_message']}")
    print(f"处理日志: {' -> '.join(result['processing_log'])}")

## 6. 实践：智能客服路由系统

让我们构建一个实际的智能客服路由系统，展示条件边的实际应用：

In [ ]:
# 客服路由状态
class CustomerServiceState(TypedDict):
    customer_message: str
    intent: str
    priority: int
    department: str
    response: str
    satisfaction_score: Optional[int]
    escalation_level: int
    conversation_history: List[str]

# 创建客服路由图
service_graph = StateGraph(CustomerServiceState)

def analyze_customer_intent(state: CustomerServiceState) -> CustomerServiceState:
    """分析客户意图"""
    message = state["customer_message"]
    print(f"🔍 分析客户消息: {message}")
    
    # 简单的意图识别逻辑
    if any(word in message.lower() for word in ['退款', '退货', 'refund', 'return']):
        intent = "refund_request"
        priority = 8
    elif any(word in message.lower() for word in ['投诉', '不满意', 'complaint', 'dissatisfied']):
        intent = "complaint"
        priority = 9
    elif any(word in message.lower() for word in ['技术', '故障', 'bug', 'error', 'technical']):
        intent = "technical_support"
        priority = 7
    elif any(word in message.lower() for word in ['账单', '费用', 'bill', 'charge', 'payment']):
        intent = "billing"
        priority = 6
    elif any(word in message.lower() for word in ['咨询', '询问', 'question', 'ask', 'how']):
        intent = "inquiry"
        priority = 3
    else:
        intent = "general"
        priority = 2
    
    print(f"🎯 识别意图: {intent}, 优先级: {priority}")
    
    return {
        "intent": intent,
        "priority": priority,
        "escalation_level": 0,
        "conversation_history": [f"客户: {message}", f"系统: 已识别意图为{intent}"]
    }

def route_to_department(state: CustomerServiceState) -> str:
    """路由到相应部门"""
    intent = state["intent"]
    priority = state["priority"]
    
    # 高优先级直接转人工
    if priority >= 8:
        print(f"🚨 高优先级问题 ({priority})，转人工客服")
        return "human_agent"
    
    # 根据意图路由
    routing_map = {
        "technical_support": "technical_team",
        "billing": "billing_team",
        "inquiry": "auto_response",
        "general": "auto_response"
    }
    
    department = routing_map.get(intent, "general_support")
    print(f"📍 路由到: {department}")
    
    return department

def auto_response(state: CustomerServiceState) -> CustomerServiceState:
    """自动回复"""
    intent = state["intent"]
    
    responses = {
        "inquiry": "感谢您的咨询！我们的常见问题页面可能对您有帮助。如需进一步协助，请联系人工客服。",
        "general": "您好！我是智能客服助手。请详细描述您遇到的问题，我会为您提供最合适的帮助。"
    }
    
    response = responses.get(intent, "感谢联系我们，正在为您查找相关信息...")
    
    print(f"🤖 自动回复: {response}")
    
    # 模拟客户满意度
    satisfaction = random.choice([3, 4, 5, 6, 7])
    
    return {
        "department": "auto_service",
        "response": response,
        "satisfaction_score": satisfaction,
        "conversation_history": state["conversation_history"] + [f"系统: {response}"]
    }

def technical_support(state: CustomerServiceState) -> CustomerServiceState:
    """技术支持"""
    print(f"🔧 技术团队处理中...")
    
    responses = [
        "我们的技术团队正在分析您的问题，预计24小时内回复。",
        "请尝试重启设备，如果问题仍然存在，我们将安排专人跟进。",
        "技术问题已记录，我们会在2个工作日内提供解决方案。"
    ]
    
    response = random.choice(responses)
    satisfaction = random.choice([6, 7, 8, 9])
    
    return {
        "department": "technical",
        "response": response,
        "satisfaction_score": satisfaction,
        "conversation_history": state["conversation_history"] + [f"技术支持: {response}"]
    }

def billing_support(state: CustomerServiceState) -> CustomerServiceState:
    """账单支持"""
    print(f"💳 账单团队处理中...")
    
    responses = [
        "我们已经为您查询了账单详情，相关信息将发送到您的邮箱。",
        "账单问题已核实，我们会在3个工作日内处理完毕。",
        "感谢您的反馈，账单调整将在下一个计费周期生效。"
    ]
    
    response = random.choice(responses)
    satisfaction = random.choice([7, 8, 9])
    
    return {
        "department": "billing",
        "response": response,
        "satisfaction_score": satisfaction,
        "conversation_history": state["conversation_history"] + [f"账单支持: {response}"]
    }

def human_agent(state: CustomerServiceState) -> CustomerServiceState:
    """人工客服"""
    print(f"👨‍💼 转接人工客服...")
    
    response = "您好！我是人工客服，感谢您的耐心等待。我会仔细处理您的问题，确保得到满意的解决方案。"
    satisfaction = random.choice([8, 9, 10])
    
    return {
        "department": "human_service",
        "response": response,
        "satisfaction_score": satisfaction,
        "conversation_history": state["conversation_history"] + [f"人工客服: {response}"]
    }

def check_satisfaction(state: CustomerServiceState) -> str:
    """检查满意度并决定后续动作"""
    satisfaction = state["satisfaction_score"]
    escalation_level = state["escalation_level"]
    
    if satisfaction <= 5 and escalation_level < 2:
        print(f"😞 满意度较低 ({satisfaction}/10)，升级处理")
        return "escalate"
    else:
        print(f"😊 处理完成，满意度: {satisfaction}/10")
        return "complete"

def escalate_issue(state: CustomerServiceState) -> CustomerServiceState:
    """升级问题"""
    escalation_level = state["escalation_level"] + 1
    print(f"⬆️ 问题升级到第 {escalation_level} 级")
    
    return {
        "escalation_level": escalation_level,
        "conversation_history": state["conversation_history"] + [f"系统: 问题已升级到第{escalation_level}级"]
    }

def complete_service(state: CustomerServiceState) -> CustomerServiceState:
    """完成服务"""
    print(f"✅ 客服流程完成")
    
    return {
        "conversation_history": state["conversation_history"] + ["系统: 感谢使用我们的客服服务！"]
    }

In [ ]:
# 构建客服路由系统
service_graph.add_node("analyze", analyze_customer_intent)
service_graph.add_node("auto_response", auto_response)
service_graph.add_node("technical_team", technical_support)
service_graph.add_node("billing_team", billing_support)
service_graph.add_node("human_agent", human_agent)
service_graph.add_node("escalate", escalate_issue)
service_graph.add_node("complete", complete_service)

service_graph.set_entry_point("analyze")

# 分析后路由到不同部门
service_graph.add_conditional_edges(
    "analyze",
    route_to_department,
    {
        "auto_response": "auto_response",
        "technical_team": "technical_team",
        "billing_team": "billing_team",
        "human_agent": "human_agent"
    }
)

# 每个处理节点后检查满意度
for node in ["auto_response", "technical_team", "billing_team", "human_agent"]:
    service_graph.add_conditional_edges(
        node,
        check_satisfaction,
        {
            "escalate": "escalate",
            "complete": "complete"
        }
    )

# 升级后转人工
service_graph.add_edge("escalate", "human_agent")
service_graph.add_edge("complete", END)

# 编译
service_app = service_graph.compile()

# 测试不同类型的客户问题
test_messages = [
    "我想退货，这个商品不符合描述",
    "账单上有一笔我不认识的费用",
    "网站登录不了，一直显示错误",
    "你们的客服态度太差了，我要投诉",
    "请问你们的营业时间是什么时候？"
]

for i, message in enumerate(test_messages):
    print(f"\n{'='*15} 客户问题 {i+1} {'='*15}")
    print(f"客户消息: {message}")
    print("")
    
    result = service_app.invoke({"customer_message": message})
    
    print(f"\n📋 处理结果:")
    print(f"意图: {result['intent']}")
    print(f"部门: {result.get('department', 'N/A')}")
    print(f"优先级: {result['priority']}/10")
    print(f"满意度: {result.get('satisfaction_score', 'N/A')}/10")
    print(f"升级次数: {result['escalation_level']}")
    print(f"最终回复: {result.get('response', 'N/A')}")

## 7. 练习题

### 练习 1：实现工作流审批系统
创建一个多级审批工作流，根据申请金额和类型路由到不同的审批路径。

In [ ]:
# 在这里实现工作流审批系统
class ApprovalState(TypedDict):
    # 定义你的状态结构
    pass

# 创建并测试你的审批工作流

### 练习 2：实现重试机制
创建一个带有智能重试逻辑的任务处理系统，根据错误类型和重试次数决定不同的处理策略。

In [ ]:
# 在这里实现重试机制


## 总结

在本课中，我们深入学习了条件边和决策逻辑：

1. **条件边基础**
   - 决策函数的编写
   - 路径映射机制
   - 简单的二分支逻辑

2. **多路分支决策**
   - 复杂的多分支路由
   - 基于多个条件的决策
   - 动态路径选择

3. **循环和递归**
   - 条件循环的实现
   - 递归结构的设计
   - 循环终止条件

4. **状态机模式**
   - 复杂状态转换
   - 业务流程建模
   - 错误处理和重试

5. **实际应用**
   - 客服路由系统
   - 订单处理流程
   - 智能决策逻辑

## 最佳实践

1. **清晰的决策逻辑**：确保决策函数逻辑清晰，易于理解和维护
2. **完整的路径覆盖**：确保所有可能的决策结果都有对应的处理路径
3. **适当的循环控制**：避免无限循环，设置合理的终止条件
4. **错误处理**：在决策函数中处理异常情况
5. **性能考虑**：避免过于复杂的决策逻辑影响执行性能

## 下一课预告

在下一课中，我们将学习：
- 工具调用和集成
- ToolNode 的使用
- Agent 执行器的构建
- 工具调用的状态管理
- 错误处理和重试机制